# Bioinformatics 101

In this notebook, we will manipulate gene expression data (RNA seq). The data is from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE183947. This notebook is created from following [this Youtube playlist](https://www.youtube.com/playlist?list=PLJefJsd1yfhbIhblS-85alaFsPdU00DaA) by Bioinformagician.

## Importing libraries and getting data

We set print options because jupyter notebook cuts columns off short.

In [1]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)

In [2]:
library(dplyr)
library(tidyverse)
library(GEOquery)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ───────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ─────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(GEOquery): there is no package called ‘GEOquery’


In [ ]:
dat <- read.csv(file = '../data/GSE183947_fpkm.csv')

In [ ]:
dim(dat)

On the website that provided the data, we can read more about it. The data consists of 30 samples with breast cancer and 30 samples without cancer. 

Looking at the data, it has 20246 genes and 60 cells. Each value represents an expression of genes normalized with [FPKM](https://docs.gdc.cancer.gov/Encyclopedia/pages/FPKM/#:~:text=Description,total%20number%20of%20mapped%20reads.) (Fragments Per Kilobase of transcript per Million mapped reads). 

In [ ]:
head(dat)

After looking at the data, we get metadata to find out which ones are cancer cells or not. We can easily get the data using `getGEO` to download it.

In [ ]:
gse <- getGEO(GEO = 'GSE183947')

In [ ]:
gse

From this, we can get the metadata.

In [ ]:
phenoData(gse[[1]])

In [ ]:
metadata <- pData(phenoData(gse[[1]]))
head(metadata, 1)

There are many columns, but we only care about 1st, 10th, 11th, and 17th columns.

In [ ]:
metadata.subset <- select(metadata, c(1,10,11,17))

In [ ]:
metadata.subset[1,]

In [ ]:
head(metadata.subset)

## Using pipe

We want to change the names of columns and values because they are horrible. What do characteristics_ch1 and characteristics_ch1.1 mean? We can change them by using pipes (`%>%`). By putting `%>%` at the end each line, the result of each line is passed to the following line. 

Here are the functions used in the pipes:
  - `select` selects columns from `metadata`.
  - `rename` renames column names.
  - `mutate` changes the data from the column.
  - `gsub` replaces the pattern. In this case, it replaces `"tissue: "` into `""`, which means deleting the matching string from `tissue` column.
  - `head` returns top 6 rows.

In [ ]:
metadata %>%
    select(1,10,11,17) %>%
    rename(tissue = characteristics_ch1) %>%
    rename(metastasis = characteristics_ch1.1) %>%
    mutate(tissue = gsub('tissue: ', '', tissue)) %>%
    mutate(metastasis = gsub('metastasis: ', '', metastasis)) %>%
    head()

After checking that it has the right format, we assign it to `metadata.modified`.

In [ ]:
metadata.modified <- metadata %>%
    select(1,10,11,17) %>%
    rename(tissue = characteristics_ch1) %>%
    rename(metastasis = characteristics_ch1.1) %>%
    mutate(tissue = gsub("tissue: ", "", tissue)) %>%
    mutate(metastasis = gsub("metastasis: ", "", metastasis))

In [ ]:
metadata.modified[1:3,]

Right now, our data is in a wide format, which means samples run through columns. We will reshape data into a long format, which is easier to work with. We will use `gather` function to reshape the data by providing key and value. `-gene` leaves the gene column alone.

In [ ]:
head(dat)

In [ ]:
dat.long <- dat %>%
    rename(gene = X) %>%
    gather(key = 'samples', value = 'FPKM', -gene)

In [ ]:
head(dat.long)

In [ ]:
dim(dat.long)

Now, we will join dat.long and metadata.modified so we have gene expression and cancer value in one dataframe. We use `left_join` which adds additional columns on the left dataframe (`dat.long` in this case). We take a look at the result of the pipeline with `head` then assigns the result into `dat.long`.

In [ ]:
dat.long %>%
    left_join(., metadata.modified, by = c('samples' = 'description')) %>%
    head()

In [ ]:
dat.long <- dat.long %>%
    left_join(., metadata.modified, by = c('samples' = 'description'))

In [ ]:
head(dat.long)

## Explore data

We can explore the data. Since we are interested in breast cancer, we want to look at `BRCA1` and `BRCA2` genes, which are tumor suppressor genes. 

Pipe line:
  - `filter` filters rows by values.
  - `group_by` groups by gene and tissue columns so that we can calculate values.
  - `summarise` makes a summary by calculating mean and median.
  - `arrange` sorts by values. `-mean_FPKM` sorts by descending order.
  - `head` provides us with an output.

In [ ]:
dat.long %>%
    filter(gene == 'BRCA1' | gene == 'BRCA2') %>%
    group_by(gene, tissue) %>%
    summarise(mean_FPKM = mean(FPKM),
              median_FPKM = median(FPKM)) %>%
    arrange(-mean_FPKM) %>%
    head()